https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/

In [ ]:
from langchain_community.graphs import Neo4jGraph

url = "neo4j://localhost:7687"
username = "neo4j"
password = "anatomy-exodus-pigment-colombo-octopus-1216"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [26]:
from pathlib import Path
markdown_document = Path('data/test.md').read_text()
markdown_document

'[ 본문 내용으로 건너뛰기 ](https://www.lguplus.com/mobile/device/phone#cSection)\n오늘 하루 그만보기\n닫기\n상단 정보영역\n# [LG U+](https://www.lguplus.com/mobile/device/phone)\n  * [ 개인 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n  * [ 기업 ](https://www.lguplus.com/mobile/device/phone)\n  * [ 소상공인 ](https://www.lguplus.com/mobile/device/phone)\n\n\n  * [ 모바일 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n    * 모바일 기기\n      * [ 휴대폰](https://www.lguplus.com/mobile/device/phone)\n      * [ 중고폰](https://www.lguplus.com/mobile/device/phone)\n      * [ 자급제폰](https://www.lguplus.com/mobile/device/phone)\n      * [ 자급제폰 등록](https://www.lguplus.com/mobile/device/phone)\n      * [ 태블릿](https://www.lguplus.com/mobile/device/phone)\n      * [ 스마트 워치](https://www.lguplus.com/mobile/device/phone)\n      * [ 휴대폰 지원금](https://www.lguplus.com/mobile/device/phone)\n    * 모바일 요금제\n      * [ 5G/LTE](https://www.lguplus.com/mobile/device/phone)\n      * [ 온라인 단독 요금제](https://www.lguplus.com/mobile/device/phone)\

In [28]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(metadata={}, page_content='[ 본문 내용으로 건너뛰기 ](https://www.lguplus.com/mobile/device/phone#cSection)\n오늘 하루 그만보기\n닫기\n상단 정보영역'),
 Document(metadata={'Header 1': '[LG U+](https://www.lguplus.com/mobile/device/phone)'}, page_content='# [LG U+](https://www.lguplus.com/mobile/device/phone)\n* [ 개인 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n* [ 기업 ](https://www.lguplus.com/mobile/device/phone)\n* [ 소상공인 ](https://www.lguplus.com/mobile/device/phone)  \n* [ 모바일 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n* 모바일 기기\n* [ 휴대폰](https://www.lguplus.com/mobile/device/phone)\n* [ 중고폰](https://www.lguplus.com/mobile/device/phone)\n* [ 자급제폰](https://www.lguplus.com/mobile/device/phone)\n* [ 자급제폰 등록](https://www.lguplus.com/mobile/device/phone)\n* [ 태블릿](https://www.lguplus.com/mobile/device/phone)\n* [ 스마트 워치](https://www.lguplus.com/mobile/device/phone)\n* [ 휴대폰 지원금](https://www.lguplus.com/mobile/device/phone)\n* 모바일 요금제\n* [ 5G/LTE](https://www.lguplus.com/mobile/device/

In [30]:
# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 1000
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
splits

[Document(metadata={}, page_content='[ 본문 내용으로 건너뛰기 ](https://www.lguplus.com/mobile/device/phone#cSection)\n오늘 하루 그만보기\n닫기\n상단 정보영역'),
 Document(metadata={'Header 1': '[LG U+](https://www.lguplus.com/mobile/device/phone)'}, page_content='# [LG U+](https://www.lguplus.com/mobile/device/phone)\n* [ 개인 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n* [ 기업 ](https://www.lguplus.com/mobile/device/phone)\n* [ 소상공인 ](https://www.lguplus.com/mobile/device/phone)  \n* [ 모바일 ](https://www.lguplus.com/mobile/device/phone "선택됨")\n* 모바일 기기\n* [ 휴대폰](https://www.lguplus.com/mobile/device/phone)\n* [ 중고폰](https://www.lguplus.com/mobile/device/phone)\n* [ 자급제폰](https://www.lguplus.com/mobile/device/phone)\n* [ 자급제폰 등록](https://www.lguplus.com/mobile/device/phone)\n* [ 태블릿](https://www.lguplus.com/mobile/device/phone)\n* [ 스마트 워치](https://www.lguplus.com/mobile/device/phone)\n* [ 휴대폰 지원금](https://www.lguplus.com/mobile/device/phone)\n* 모바일 요금제\n* [ 5G/LTE](https://www.lguplus.com/mobile/device/

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o"
)

llm_transformer = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    # node_properties=["born_year"]
)

graph_documents = llm_transformer.convert_to_graph_documents(splits)
graph_documents

In [17]:
graph.add_graph_documents(graph_documents)